In [61]:
using Pkg
Pkg.activate("..")
Pkg.instantiate()
using JPEC, Plots

  Activating project at `~/Projects/Perturbed-Equilibrium/JPEC.worktrees/vacuum_julia`


In [62]:
function fouran!(
    gij::Matrix{Float64},
    gil::Matrix{Float64},
    cs::Matrix{Float64},
    m00::Int,
    l00::Int,
    lmin::Vector{Int},
    lmax::Vector{Int},
    mth::Int
)
    # Compute jmax1 like Fortran
    jmax1 = lmax[1] - lmin[1] + 1

    # Zero out relevant gil block
    for l1 in 1:jmax1
        for i in 1:mth
            gil[m00 + i, l00 + l1] = 0.0
        end
    end

    # Accumulate with ll offset (critical to match Fortran)
    for l1 in 1:jmax1
        ll = l1 - 1 + lmin[1]
        for j in 1:mth
            for i in 1:mth
                gil[m00 + i, l00 + l1] += cs[j, l1] * gij[i, j]
            end
        end
    end

    return nothing
end

fouran! (generic function with 1 method)

In [63]:
function foranv!(gil::Matrix{Float64}, gll::Matrix{Float64}, cs::Matrix{Float64},
                 m00::Int, l00::Int, mth::Int, lmin::Int, lmax::Int,
                 dth::Float64, twopi::Float64)

    jmax1 = lmax - lmin + 1

    # Zero out gll block
    for l1 in 1:jmax1
        for l2 in 1:jmax1
            gll[l2, l1] = 0.0
        end
    end

    # Main accumulation (note: gll[l2, l1], not gll[l1, l2])
    for l1 in 1:jmax1
        for l2 in 1:jmax1
            for i in 1:mth
                gll[l2, l1] += dth * cs[i, l2] * gil[m00+i, l00+l1] * twopi
            end
        end
    end

    return gll
end


foranv! (generic function with 3 methods)

In [64]:
function test_fourier()
    # Parameters
    mtheta = 200
    mthvac = 900
    mpert  = 34

    # After defglo/global_alloc
    nths  = 905
    nths2 = 1810
    nfm   = 34
    nfm2  = 68

    m00 = 0
    l00 = 34

    # Define lmin/lmax like in arrays.f (dummy example: from 1 to nfm)
    lmin = [1]
    lmax = [nfm]

    # Allocate arrays
    gij = zeros(Float64, nths, nths)
    gil = zeros(Float64, nths2, nfm2)
    cs  = zeros(Float64, nths, nfm)
    x   = zeros(Float64, nths)

    # Allocate gll for foranv test
    gll = zeros(Float64, nfm, nfm)

    # Fill x(i) = 6.28 / i
    for i in 1:nths
        x[i] = 6.28 / i
    end

    # Fill gij(i,j) and cs(i,j)
    for i in 1:nths
        for j in 1:nths
            gij[i,j] = sin(x[i] + x[j])
        end
        for j in 1:nfm
            cs[i,j] = (sin(j * x[i]))^2
        end
    end

    # Call your Fourier routine
    fouran!(gij, gil, cs, m00, l00, lmin, lmax, nths)

    println("gil(1:5,1:5) submatrix starting at (109+1,33+1):")
    for i in 1:5
        println([gil[109+i, 33+j] for j in 1:5])
    end

    mth = nths
    twopi = 2π
    dth = twopi / mth
    gll = zeros(Float64, nfm, nfm)

    foranv!(gil, gll, cs, m00, l00, mth, lmin[1], lmax[1], dth, twopi)

    println("gll(1:5,1:5) matrix:")
    for i in 1:5
        println([gll[i,j] for j in 1:5])
    end

end

test_fourier()


gil(1:5,1:5) submatrix starting at (109+1,33+1):
[0.0, 5.758492623190696, 8.519982775454418, 10.099787086819953, 11.7302450407185]
[0.0, 5.756099490628109, 8.512536094163833, 10.087290470562392, 11.713017282668279]
[0.0, 5.753747610409246, 8.505220197384235, 10.075014410727995, 11.696094147093449]
[0.0, 5.75143592692491, 8.49803167245637, 10.062953123235024, 11.679467630240492]
[0.0, 5.749163420208127, 8.490967224332074, 10.051101024364383, 11.663130006138648]
gll(1:5,1:5) matrix:
[2.3789313086264325, 5.239419792794563, 7.826355667110261, 10.284593399378773, 12.795304525965333]
[5.239419792794563, 10.009696479612979, 13.986576309804287, 17.813106486393192, 21.751528156905444]
[7.8263556671102545, 13.986576309804267, 18.84118428634243, 23.555201297958067, 28.43403506496631]
[10.284593399378766, 17.81310648639312, 23.555201297958053, 29.161845740032398, 34.9837133827433]
[12.795304525965348, 21.75152815690543, 28.434035064966384, 34.98371338274334, 41.80016994126332]
